Sambungkan ke Gdrive lalu upload dataset ke dalam folder bernama dataset_cakrawala untuk mengakses dataset (jika ingin mencoba script untuk membuat model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Jalankan pip terlebih dahulu

In [ ]:
!pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


model 1

In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

# 1. Load Data from txt files
def load_data(folder_path):
    texts = []
    labels = []
    for label in ['human-written', 'ai-generated']:
        path = os.path.join(folder_path, label)
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(1 if label == 'ai-generated' else 0)
    return texts, labels

data, labels = load_data('/content/drive/MyDrive/dataset_cakrawala')

# 2. Tokenize text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)  # Adjust num_words as needed
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=100)  # Adjust maxlen as needed

# Convert labels to numpy array
y = np.array(labels)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Define and compile the Keras model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    GlobalAveragePooling1D(),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training with early stopping if accuracy does not improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, mode='max', verbose=1)

# 5. Train the model
history = model.fit(X_train, y_train, epochs=10000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 6. Save the Keras model in HDF5 format (.h5)
model.save('best_trained_model.h5')

# 7. Convert the Keras model to TensorFlow.js format
tfjs.converters.save_keras_model(model, 'tfjs_model')


Epoch 1/10000
44/44 [==============================] - 2s 29ms/step - loss: 0.6891 - accuracy: 0.6329 - val_loss: 0.6835 - val_accuracy: 0.7138
Epoch 2/10000
44/44 [==============================] - 1s 20ms/step - loss: 0.6691 - accuracy: 0.7957 - val_loss: 0.6676 - val_accuracy: 0.6855
Epoch 3/10000
44/44 [==============================] - 1s 19ms/step - loss: 0.6366 - accuracy: 0.8114 - val_loss: 0.6445 - val_accuracy: 0.6938
Epoch 4/10000
44/44 [==============================] - 1s 19ms/step - loss: 0.5935 - accuracy: 0.8314 - val_loss: 0.6189 - val_accuracy: 0.7072
Epoch 5/10000
44/44 [==============================] - 1s 19ms/step - loss: 0.5446 - accuracy: 0.8479 - val_loss: 0.5955 - val_accuracy: 0.7155
Epoch 6/10000
44/44 [==============================] - 1s 13ms/step - loss: 0.4959 - accuracy: 0.8671 - val_loss: 0.5747 - val_accuracy: 0.7205
Epoch 7/10000
44/44 [==============================] - 1s 14ms/step - loss: 0.4497 - accuracy: 0.8807 - val_loss: 0.5585 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


model 2 (used)

In [ ]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

# 1. Load Data from txt files
def load_data(folder_path):
    texts = []
    labels = []
    for label in ['human-written', 'ai-generated']:
        path = os.path.join(folder_path, label)
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(1 if label == 'ai-generated' else 0)  # Updated labels to binary values
    return texts, labels

data, labels = load_data('/content/drive/MyDrive/dataset_cakrawala') #update path sesuai path dataset

# 2. Convert text to numerical features using TF-IDF
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.7)
X = tfidf.fit_transform(data).toarray()  # Convert to numpy array

# Convert labels to numpy array
y = np.array(labels)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Define and compile the Keras model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training with early stopping if accuracy does not improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, mode='max', verbose=1)

# 5. Train the model
history = model.fit(X_train, y_train, epochs=10000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 6. Save the Keras model in HDF5 format (.h5)
model.save('training_model_terbaik.h5')

Epoch 1/10000
44/44 [==============================] - 5s 96ms/step - loss: 0.6412 - accuracy: 0.7064 - val_loss: 0.5813 - val_accuracy: 0.7637
Epoch 2/10000
44/44 [==============================] - 3s 64ms/step - loss: 0.4209 - accuracy: 0.9171 - val_loss: 0.4964 - val_accuracy: 0.7870
Epoch 3/10000
44/44 [==============================] - 2s 56ms/step - loss: 0.2439 - accuracy: 0.9714 - val_loss: 0.4586 - val_accuracy: 0.7987
Epoch 4/10000
44/44 [==============================] - 2s 50ms/step - loss: 0.1345 - accuracy: 0.9929 - val_loss: 0.4433 - val_accuracy: 0.7937
Epoch 5/10000
44/44 [==============================] - 3s 58ms/step - loss: 0.0768 - accuracy: 0.9979 - val_loss: 0.4409 - val_accuracy: 0.7903
Epoch 6/10000
44/44 [==============================] - 2s 56ms/step - loss: 0.0477 - accuracy: 0.9993 - val_loss: 0.4438 - val_accuracy: 0.7887
Epoch 7/10000
44/44 [==============================] - 4s 98ms/step - loss: 0.0320 - accuracy: 1.0000 - val_loss: 0.4462 - val_accuracy:

Script untuk memproses model kemudian print di terminal.

In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import re

# Import and define TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Input user text for prediction
user_input = input("Masukkan teks: ")

# Preprocess the user input similar to the training data
user_input_features = tokenizer.texts_to_sequences([user_input])
user_input_features = tf.keras.preprocessing.sequence.pad_sequences(user_input_features, maxlen=100)

# Load the trained model for prediction
loaded_model = load_model('best_trained_model.h5')

# Predict whether the input is AI-generated or human-written
prediction = loaded_model.predict(user_input_features)
ai_generated_percentage = prediction[0][0] * 100
human_written_percentage = (1 - prediction[0][0]) * 100

# Determine if input is AI-generated or human-written based on a threshold
threshold = 0.5
if prediction > threshold:
    input_type = "AI-generated"
else:
    input_type = "Human-written"

# Display the percentages of AI-generated and human-written text
print(f"Persentase teks hasil AI: {ai_generated_percentage:.2f}%")
print(f"Persentase teks buatan manusia: {human_written_percentage:.2f}%")

# Display the conclusion based on the threshold
print(f"Teks cenderung buatan: {input_type}")

# Find sentences with high AI-generated probability
sentences = re.split(r'(?<=[.!?]) +', user_input)
ai_generated_sentences = []

for sentence in sentences:
    sentence_features = tokenizer.texts_to_sequences([sentence])
    sentence_features = tf.keras.preprocessing.sequence.pad_sequences(sentence_features, maxlen=100)
    sentence_prediction = loaded_model.predict(sentence_features)
    if sentence_prediction > threshold:  # Adjusted variable name to sentence_prediction
        ai_generated_sentences.append(sentence)

# Display AI-generated sentences
if ai_generated_sentences:
    print("Kalimat yang terindikasi buatan AI:")
    for sentence in ai_generated_sentences:
        print(f"- {sentence}")


Masukkan teks: In 1950, the English mathematician Alan Turing developed a series of tests to distinguish between robots and people. The questions, called ‘The Turing Test,’ can be used with some accuracy to ascertain whether, when speaking to a character online, they are a real human being or a highly intelligent software programme. This year, those questions gained an entirely new relevance. An AI Language Model, called ChatGTP, became a viral sensation. You can pose the most fascinating questions to this robot, and it will answer them as if you were speaking to a real human being. It can have conversations and play word games and share thoughts on current events. It can even write a half-decent sermon. But there were some questions it couldn’t answer. It still cannot pass the Turing Test. Tech experts promise that, very soon, it might. But, for now, there are certain things it cannot find adequate responses for.
1/1 [==============================] - 0s 53ms/step
Persentase teks hasi